In [1]:
from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, \
    squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import inv
from scipy.stats import chi2
from pandas import read_csv
import statsmodels.api as sm
import os


In [2]:
os.chdir("/Users/mandarphatak/Downloads/")

## Case 1:- CFNAI index

In [7]:
data=read_csv("ff3_sentiments.csv")
data

,date,rf,mkt_excess,smb,hml,cfnai,cfnai_mkt,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,0.15,0.002145,0.025895,0.021216,-0.002137,-0.014098,-0.002741,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,0.45,0.020430,0.052652,0.053536,0.040117,0.075273,0.035722,0.024941,0.069021,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,0.01,0.000343,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028173
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,0.81,-0.022356,-0.055374,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059923,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,0.17,0.003213,0.018897,-0.000566,0.005434,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,-0.43,-0.013803,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,0.17,-0.004063,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-0.14,0.007336,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,0.02,-0.000638,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [9]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','cfnai','cfnai_mkt']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree


In [11]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [12]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [13]:
# We increase by sentiments and augmented factor. so 5  as we are using three factors
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)
print(J)
print(Jpval)

### We dont rejec the Jtest, the null hypthesis that alpha is equal to zero is satisfied in our case

13.599020038999207
0.2559804142549069


In [23]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB      HML   CFNAI   CFNAI_MKT')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}   {2:0.4f}  {3:0.4f}  {4:0.4f}'.format(arp[0], arp[1], arp[2], arp[3], arp[4]))
print('Std. Err.  {0:0.4f}    {1:0.4f}      {2:0.4f}  {3:0.4f}   {4:0.4f} '.format(arpSE[0], arpSE[1], arpSE[2],arpSE[3],arpSE[4]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CFNAI) Beta(CFNAI_MKT)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2], b[3],b[4]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(s[0], s[1], s[2], s[3],s[4],s[5]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(t[0], t[1], t[2], t[3], t[4],t[5]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB      HML   CFNAI   CFNAI_MKT
---------------------------------------------------------------
Premia     0.0943    -0.0476   -0.0449  -8.8626  -0.5792
Std. Err.  0.0105    0.0123      0.0091  4.3740   0.2446 



J-test:   13.5990
P-value:   0.2560
Size: 1, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CFNAI) Beta(CFNAI_MKT)
Coefficients:    -0.0020      0.9191     -0.0092     -0.1087    -0.0026      0.0324
Std Err.          0.0024      0.0702      0.0863      0.0684     0.0058      0.0684
T-stat           -0.8376     13.0951     -0.1062     -1.5882    -0.4466      0.4742

Size: 2, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CFNAI) Beta(CFNAI_MKT)
Coefficients:     0.0002      1.1538      0.2534     -0.1053     0.0010     -0.0261
Std Err.          0.0015      0.0472      0.0642      0.0498     0.0027      0.0451
T-stat            0.1570     24.4495      3.9496     -2.1131     0.3691     -0.5792

Size: 3, V

In [ ]:
## We can clearly see that Beta(mkt) is dominant and significat for all the ETFs, 

## Case 2:- UMCSENT Index:-


In [25]:
data = read_csv("ff3_sentiments1.csv")
data

,date,rf,mkt_excess,smb,hml,umcsent,umcsent_mkt,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,-1.7,-0.02431,0.025895,0.021216,-0.002137,-0.014098,-0.002741,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,-2.5,-0.11350,0.052652,0.053536,0.040117,0.075273,0.035722,0.024941,0.069021,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,1.1,0.03773,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028173
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,4.3,-0.11868,-0.055374,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059923,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,-1.6,-0.03024,0.018897,-0.000566,0.005434,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,5.2,0.16692,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,7.3,-0.17447,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-2.1,0.11004,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-1.5,0.04785,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [26]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','umcsent','umcsent_mkt']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [27]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [28]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [29]:
# We increase by 6 as we are using three factors , sentiment and augmented factor
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)
print(J)
print(Jpval)

14.698648843546401
0.19671273999392946


In [ ]:
### So we have the similar outcome as before we are accepting the J test

In [30]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB      HML   UMCSENT   UMCSENT_MKT')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}   {2:0.4f}  {3:0.4f}  {4:0.4f}'.format(arp[0], arp[1], arp[2], arp[3], arp[4]))
print('Std. Err.  {0:0.4f}    {1:0.4f}      {2:0.4f}  {3:0.4f}   {4:0.4f} '.format(arpSE[0], arpSE[1], arpSE[2],arpSE[3],arpSE[4]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(UMCSENT) Beta(UMCSENT_MKT)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2], b[3],b[4]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(s[0], s[1], s[2], s[3],s[4],s[5]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(t[0], t[1], t[2], t[3], t[4],t[5]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB      HML   UMCSENT   UMCSENT_MKT
---------------------------------------------------------------
Premia     0.0993    -0.0662   -0.0292  -17.2610  -0.7354
Std. Err.  0.0104    0.0121      0.0102  9.0829   0.2569 



J-test:   14.6986
P-value:   0.1967
Size: 1, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(UMCSENT) Beta(UMCSENT_MKT)
Coefficients:    -0.0015      0.9017      0.0026     -0.1250    -0.0002      0.0085
Std Err.          0.0019      0.0527      0.0823      0.0640     0.0005      0.0087
T-stat           -0.7632     17.0944      0.0314     -1.9516    -0.3219      0.9691

Size: 2, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(UMCSENT) Beta(UMCSENT_MKT)
Coefficients:    -0.0001      1.1632      0.2484     -0.0939    -0.0001     -0.0131
Std Err.          0.0013      0.0381      0.0647      0.0487     0.0003      0.0067
T-stat           -0.0866     30.5641      3.8390     -1.9281    -0.2087     -1.952

In [ ]:
### We can clearly see that Beta(mkt) is highly significant 

## Case 3: PMI Index


In [32]:
data=read_csv('ff3_sentiment2.csv')
data

,date,rf,mkt_excess,smb,hml,pmi,pmi_mkt,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,57.4,0.82082,0.025895,0.021216,-0.002137,-0.014098,-0.002741,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,56.3,2.55602,0.052652,0.053536,0.040117,0.075273,0.035722,0.024941,0.069021,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,56.2,1.92766,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028173
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,57.2,-1.57872,-0.055374,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059923,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,56.8,1.07352,0.018897,-0.000566,0.005434,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,46.3,1.48623,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,49.0,-1.17110,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,47.9,-2.50996,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,49.8,-1.58862,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [33]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','pmi','pmi_mkt']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [34]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [35]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [36]:
# We increase by 6 as we are using three factors , sentiment and augmented factor
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)
print(J)
print(Jpval)

9.28091408645562
0.5959800576501072


In [ ]:
### We donot reject the J test and accept the null H0: Alpha is equal to zero


In [37]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB      HML   PMI   PMI_MKT')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}   {2:0.4f}  {3:0.4f}  {4:0.4f}'.format(arp[0], arp[1], arp[2], arp[3], arp[4]))
print('Std. Err.  {0:0.4f}    {1:0.4f}      {2:0.4f}  {3:0.4f}   {4:0.4f} '.format(arpSE[0], arpSE[1], arpSE[2],arpSE[3],arpSE[4]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(PMI) Beta(PMI)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2], b[3],b[4]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(s[0], s[1], s[2], s[3],s[4],s[5]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(t[0], t[1], t[2], t[3], t[4],t[5]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB      HML   PMI   PMI_MKT
---------------------------------------------------------------
Premia     0.1043    -0.0141   -0.0600  36.4004  6.4733
Std. Err.  0.0105    0.0116      0.0089  5.8558   0.5934 



J-test:   9.2809
P-value:   0.5960
Size: 1, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(PMI) Beta(PMI)
Coefficients:     0.0401      0.5877     -0.0090     -0.0918    -0.0008      0.0062
Std Err.          0.0199      0.3967      0.0799      0.0677     0.0004      0.0075
T-stat            2.0118      1.4814     -0.1131     -1.3570    -2.1329      0.8155

Size: 2, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(PMI) Beta(PMI)
Coefficients:     0.0125      1.6426      0.2489     -0.1165    -0.0002     -0.0092
Std Err.          0.0209      1.8027      0.0619      0.0887     0.0004      0.0348
T-stat            0.5977      0.9112      4.0186     -1.3132    -0.5887     -0.2640

Size: 3, Value:1   Alpha   Beta(

In [ ]:
### In the case we can clearly see that  Beta(hml) is significat for most of the sectors, Beta(smb) is for a few. 
#surprisingly Beta(mkt) is not significat

##  Case4: Consumer Confidence Index 

In [39]:
data=read_csv('ff3_sentiments3.csv')
data

,date,rf,mkt_excess,smb,hml,cci,cci_mkt,Communications Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
0,2004-10-01,0.0011,0.0143,0.0015,-0.0022,-7.0,-0.10010,0.025895,0.021216,-0.002137,-0.014098,-0.002741,-0.028816,0.010482,0.020669,-0.020334,0.037515,0.035496
1,2004-11-01,0.0015,0.0454,0.0374,0.0141,-2.0,-0.09080,0.052652,0.053536,0.040117,0.075273,0.035722,0.024941,0.069021,0.056655,0.096553,0.039548,0.043742
2,2004-12-01,0.0016,0.0343,-0.0003,-0.0022,-3.8,-0.13034,0.036160,0.052639,0.042355,-0.028016,0.041629,0.057599,0.030434,0.026588,0.011761,0.047553,0.028173
3,2005-01-01,0.0016,-0.0276,-0.0172,0.0206,-0.3,0.00828,-0.055374,-0.038796,0.011545,0.037397,-0.030477,-0.032816,-0.030419,-0.059923,-0.032094,-0.085588,0.010587
4,2005-02-01,0.0016,0.0189,-0.0057,0.0154,10.1,0.19089,0.018897,-0.000566,0.005434,0.165533,-0.003067,0.030845,0.015217,0.005651,0.084211,0.031715,0.021798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2023-07-01,0.0045,0.0321,0.0208,0.0411,7.4,0.23754,0.059355,0.033969,0.022371,0.082913,0.060192,0.010293,0.031431,0.028610,0.039457,0.020345,0.024056
226,2023-08-01,0.0045,-0.0239,-0.0316,-0.0106,7.3,-0.17447,-0.028503,-0.020149,-0.031891,0.019468,-0.030535,-0.011725,-0.020520,-0.021833,-0.035263,-0.033779,-0.063397
227,2023-09-01,0.0043,-0.0524,-0.0251,0.0152,-10.9,0.57116,-0.035362,-0.058614,-0.045520,0.023748,-0.032034,-0.034630,-0.057756,-0.065384,-0.049823,-0.072638,-0.055950
228,2023-10-01,0.0047,-0.0319,-0.0387,0.0019,-3.1,0.09889,-0.015876,-0.051973,-0.017408,-0.053653,-0.029009,-0.039558,-0.039184,-0.017113,-0.038488,-0.036215,0.008154


In [40]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['mkt_excess', 'smb', 'hml','cci','cci_mkt']].values
riskfree = data['rf'].values
portfolios = data.iloc[:, 7:].values

# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

# Shape information
T,K = factors.shape
T,N = portfolios.shape
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

In [41]:
# Time series regressions
X = sm.add_constant(factors)
ts_res = sm.OLS(excessReturns, X).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]
avgExcessReturns = mean(excessReturns, 0)
# Cross-section regression
cs_res = sm.OLS(avgExcessReturns.T, beta.T).fit()
riskPremia = cs_res.params

In [42]:
# Moment conditions
X = sm.add_constant(factors)
p = vstack((alpha, beta))
epsilon = excessReturns - X @ p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia[None,:] @ beta
moments2 = u * beta.T
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = (X.T @ X) / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta @ beta.T
for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [43]:
# We increase by 6 as we are using three factors , sentiment and augmented factor
vcvAlpha = vcv[0:N * K + N:6, 0:N * K + N:6]
J = alpha @ inv(vcvAlpha) @ alpha.T
J = J[0, 0]
Jpval = 1 - chi2(11).cdf(J)
print(J)
print(Jpval)

15.260190026780004
0.1708924905624889


In [ ]:
### We are not rejecting the J test and thus accepting the null :- Alpha is zero

In [44]:
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print('        Annualized Risk Premia')
print('           Market       SMB      HML   CCI   CCI_MKT')
print('---------------------------------------------------------------')
print('Premia     {0:0.4f}    {1:0.4f}   {2:0.4f}  {3:0.4f}  {4:0.4f}'.format(arp[0], arp[1], arp[2], arp[3], arp[4]))
print('Std. Err.  {0:0.4f}    {1:0.4f}      {2:0.4f}  {3:0.4f}   {4:0.4f} '.format(arpSE[0], arpSE[1], arpSE[2],arpSE[3],arpSE[4]))
print('\n\n')

print('J-test:   {:0.4f}'.format(J))
print('P-value:   {:0.4f}'.format(Jpval))

i = 0
betaSE = []
for j in range(11):
    for k in range(1):
        a = alpha[i]
        b = beta[:, i]
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(a, b[0], b[1], b[2], b[3],b[4]))
        print('Std Err.      {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(s[0], s[1], s[2], s[3],s[4],s[5]))
        print('T-stat        {:>10,.4f}  {:>10,.4f}  {:>10,.4f}  {:>10,.4f} {:>10,.4f}  {:>10,.4f}'.format(t[0], t[1], t[2], t[3], t[4],t[5]))
        print('')
        i += 1

        Annualized Risk Premia
           Market       SMB      HML   CCI   CCI_MKT
---------------------------------------------------------------
Premia     0.0992    -0.0407   -0.0416  -4.1210  0.7624
Std. Err.  0.0104    0.0121      0.0089  10.3187   0.5256 



J-test:   15.2602
P-value:   0.1709
Size: 1, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)
Coefficients:    -0.0015      0.9007      0.0007     -0.1229     0.0000      0.0012
Std Err.          0.0019      0.0534      0.0830      0.0636     0.0003      0.0079
T-stat           -0.7927     16.8726      0.0087     -1.9331     0.1327      0.1500

Size: 2, Value:1   Alpha   Beta(MKT)   Beta(SMB)   Beta(HML) Beta(CCI) Beta(CCI_MKT)
Coefficients:    -0.0000      1.1595      0.2537     -0.1052     0.0002     -0.0088
Std Err.          0.0013      0.0375      0.0645      0.0489     0.0002      0.0051
T-stat           -0.0143     30.8964      3.9304     -2.1510     0.9894     -1.7445

Size: 3, Value:1   Alp

In [ ]:
## Beta(MKT) is significant for all the ETFs, foe a few Beta(SMB) is also significant.